In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

d:\doitLLM_book_test\DoitLLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='네, 감사합니다! 당신은 잘 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Czu3veniKppRmoonfKuBMzTsOIqqM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd8d7-4c22-7ac2-a062-f3e3a8ae0dad-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool #함수를 도구로 등록
def get_current_time(timezone: str, location: str)-> str:
    """
        현재 시각을 반환하는 함수

        Args:
            timezone(str) : 타임존(예:Asia/Seoul), 실제 존재해야 함
            localtion(str) : 지역명, 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now}'
    
    print(location_and_local_time)
    return location_and_local_time

In [8]:
#도구를 tools 리스트에 추가하고 tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time" : get_current_time}

#도구를 모델에 바인딩
llm_with_tools = llm.bind_tools(tools)

In [9]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다"),
    HumanMessage("부산은 몇시야?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 128, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Czu495aot2zhacJ0Y7eXIeCxMKcKZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd8d7-8158-7a40-8e8b-0284258e1d37-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_MB5ZHRoUAdep1atRVApBX6sN', 'type': 'tool_call'}], 

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2026-01-20 09:39:31


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 128, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Czu495aot2zhacJ0Y7eXIeCxMKcKZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd8d7-8158-7a40-8e8b-0284258e1d37-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_MB5ZHRoUAdep1atRVApBX6sN', 'type': 'tool_call'}]

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026년 1월 20일 09시 39분 31초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 184, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-Czu4LzvmOVSYyqWF3hqFkfgFgEJJp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd8d7-b565-7210-aa99-0a60a6785f49-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 184, 'output_tokens': 29, 'total_tokens': 213, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker : str = Field(...,title="주식 코드", description="주식 코드 (예: APPL)")
    period : str = Field(...,title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [14]:
import yfinance as yf

@tool # 함수를 도구로 변환, 랭체인에서 외부 도구로 등록하여 언어모델이 함수를 호출하고 사용할 수 있게 함
def get_yf_stock_history(stock_history_input : StockHistoryInput) -> str:
    """주식 종목의 가격 데이터를 조회하는 함수"""

    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period = stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time" : get_current_time, "get_yf_stock_history" : get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)


In [15]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나? 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 276, 'total_tokens': 347, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzuPgtVfAwIgJ82MhR9fcIAItSPqF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bd8eb-e540-7151-9e05-0b04af3875b1-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_seQmqBljXUvQCGyVIdL6YHWX', 'type': 'tool_call'}, {'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1d'}}, 'id': 'call_FshYC0A3w7C7ovcXbBOinFp8', 'type': 'tool_call'}] invalid_tool_calls=[] 

In [16]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-12-17 00:00:00-05:00 | 488.22 | 495.28 | 466.2  |  467.26 | 1.0649e+08  |           0 |              0 |\n| 2025-12-18 00:00:00-05:00 | 478.16 | 490.86 | 473.12 |  483.37 | 9.51684e+07 |           0 |              0 |\n| 2025-12-19 00:00:00-05:00 | 488.12 | 490.49 | 474.72 |  481.2  | 1.03305e+08 |           0 |              0 |\n| 2025-12-22 00:00:00-05:00 | 489.88 | 498.83 | 485.33 |  488.73 | 8.69161e+07 |           0 |              0 |\n| 2025-12-23 00:00:00-05:00 | 489.4  | 491.97 | 482.84 |  485.56 | 5.82236e+07 |           0 |              0 |\n| 2025-12-24 00:00:00-05:00 | 488.48 | 490.9  | 476.8  |  485.4  | 4.12854e+07 |           0 |              0 |\n| 2025-12-26 00:00:00-05:0

In [17]:
llm_with_tools.invoke(messages)

AIMessage(content='테슬라(TSLA)의 주가는 한 달 전과 비교해 다음과 같습니다:\n\n- **현재 주가 (2026-01-16)**: $437.50\n- **한 달 전 주가 (2025-12-16)**: 주가는 데이터가 없어 정확한 수치는 제공할 수 없으나, 최근 데이터에 따르면 주가는 $467.26에서 시작했습니다.\n\n주가는 한 달 동안 약간 하락한 것으로 보입니다. \n\n주가가 한 달 전보다 하락했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 1768, 'total_tokens': 1879, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzuYRURexwriLeU5YtIMOk3PLf28g', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bd8f4-2d3a-7b91-830e-d74fbdcec151-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1768, 'output_tokens': 111, 'total_tokens': 1879, 'input_t

In [21]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 얘기해줘")]):
    print(c.content,end = "")
                    


안녕하세요! 한국 사회의 문제점은 여러 가지가 있지만, 몇 가지 주요한 분야를 살펴보겠습니다.

1. **고용 불안정**: 특히 젊은 세대는 안정적인 일자리를 찾기 어려워하고, 비정규직 일자리의 비율이 높아지고 있습니다. 이는 경제적 불안정을 초래하고 있습니다.

2. **주거 문제**: 서울과 같은 대도시의 주택 가격 상승은 많은 사람들에게 큰 부담이 되고 있습니다. 주거비가 가처분 소득을 초과하는 경우가 많아 주택 구매가 어려워지고 있습니다.

3. **교육 경쟁**: 높은 교육열과 입시 경쟁은 학생들에게 심리적 압박을 가하고 있으며, 부모들도 이에 대한 부담을 느끼고 있습니다. 이는 전반적인 삶의 질에도 부정적인 영향을 미치고 있습니다.

4. **정신 건강**: 정신 건강 문제에 대한 인식이 높아지고 있지만, 여전히 치료를 받고 싶어도 사회적 낙인이나 실질적인 접근성 문제로 어려움을 겪는 사람들이 많습니다.

5. **성 평등**: 성별에 따른 불평등, 특히 경제적 분야에서의 성차별 문제가 여전히 존재합니다. 많은 여성들이 직장에서의 차별이나 성폭력 문제로 어려움을 겪고 있습니다.

6. **노인 문제**: 인구 고령화로 인해 노인 복지와 관련된 문제도 심각해지고 있습니다. 사회 안전망이 부족한 상황에서 노인 빈곤 문제는 심각한 상황입니다.

이 외에도 환경 문제, 인종적 다양성에 대한 이해 부족 등 다양한 문제들이 존재합니다. 한국 사회는 이러한 문제들을 해결하기 위해 여러 노력을 하고 있으며, 지속적인 논의와 변화가 필요합니다.

In [24]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다"),
    HumanMessage("부산은 지금 몇시야?")
]

response = llm_with_tools.stream(messages)

is_first = True

for chunk in response:
    print("chunk type:" , type(chunk))

    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk

    print("content:", gathered.content,"tool_call_chunk", gathered.tool_calls)

messages.append(gathered)


chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:  tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}]
chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:  tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}]
chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:  tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}]
chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:  tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}]
chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:  tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}]
chu

In [25]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2026-01-20 10:29:32


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'service_tier': 'default'}, id='lc_run--019bd903-79b6-7393-aacd-ff88c4098fc1', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 197, 'output_tokens': 23, 'total_tokens': 220, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"Busan"}', 'id': 'call_a2vqJWS4xnR0rVqCkXUqDITs', 'index': 0

In [26]:
for c in llm_with_tools.stream(messages):
    print(c.content,end='')

부산은 현재 2026년 1월 20일 10시 29분입니다.